In [ ]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import pathlib 
import numpy as np
import scipy.io as sio 
import math
import keras
# tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

### training set

In [ ]:
data_root = './dataset/cars_train'
data_root = pathlib.Path(data_root)

all_image_paths = sorted([str(path) for path in data_root.iterdir()])

len(all_image_paths)

traindata = pd.read_csv('./dataset/traindata.csv', index_col=0)

# train_paths = all_image_paths
train_labels = traindata['class'].tolist()

train_bbox = traindata[['bbox1', 'bbox2', 'bbox3', 'bbox4']].values.tolist()
train_bbox = np.array(train_bbox, dtype=np.str)

all_image_paths = np.array(all_image_paths).reshape((-1, 1))

all_image_paths = np.concatenate((all_image_paths, train_bbox), axis=1)

def preprocess_image(image, bbox):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.crop_to_bounding_box(image, bbox[1], bbox[0], bbox[3]-bbox[1],
                                          bbox[2] - bbox[0])
    image = tf.image.resize_images(image, [224, 224])
#     image = (image - 127.5)/127.5  # normalize to [-1,1] range
    image = tf.image.per_image_standardization(image)
#     print(image.math.maximum)
    return image

def load_and_preprocess_image(path):
    image = tf.read_file(path[0])
    return preprocess_image(image, tf.string_to_number(path[1:], out_type=tf.int32))

class_dir = "./cars_meta.mat"
def getclass(class_dir):
    annos = sio.loadmat(class_dir)
    _, num_class = annos['class_names'].shape
    class_label = {i:annos['class_names'][0,i][0] for i in range(num_class)}
    return class_label
class_label = getclass(class_dir)

def caption_image(image_path):
    image_rel = pathlib.Path(image_path).relative_to(data_root)
    return str(image_rel)
caption_image(all_image_paths[0][0])

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image)

label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(np.array(train_labels) - 1, tf.int64))

image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

BATCH_SIZE = 32
image_count = len(all_image_paths)
# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
# ds = ds.prefetch(buffer_size=64)
ds

In [ ]:
train_labels_np = np.array([train_labels])

In [ ]:
train_labels_np.shape

In [ ]:
all_image_path_all = np.concatenate((all_image_paths, train_labels_np.T), axis=1)

In [ ]:
all_image_path_all

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,224,224), n_channels=3,
                 n_classes=196, shuffle=True):
        'Initialization'
        self.dim = dim
        self.dim[0] = batch_size
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            with Image.open(ID[0]) as img:
                img = img.crop(box=(ID[1],ID[2],ID[3],ID[4])).resize((224,224))
                img = np.array(img)
                X[i,] = img

            # Store class
            y[i] = self.ID[-1]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

### test set

In [ ]:
testdata = pd.read_csv('./dataset/testdata.csv', index_col=0) #

test_data_root = './dataset/cars_test' # /dataset
test_data_root = pathlib.Path(test_data_root)

test_paths = sorted([str(path) for path in test_data_root.iterdir()])

len(test_paths)

test_labels = testdata['class'].tolist()

test_bbox = testdata[['bbox1', 'bbox2', 'bbox3', 'bbox4']].values.tolist()
test_bbox = np.array(test_bbox, dtype=np.str)

test_paths = np.array(test_paths).reshape((-1, 1))

test_paths = np.concatenate((test_paths, test_bbox), axis=1)

test_path_ds = tf.data.Dataset.from_tensor_slices(test_paths)
test_image_ds = test_path_ds.map(load_and_preprocess_image)

test_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(np.array(test_labels) - 1, tf.int64))

test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
test_image_label_ds = test_image_label_ds.batch(BATCH_SIZE)

# resnet

In [ ]:
# from keras.applications.inception_v3 import InceptionV3
# from keras.preprocessing import image
# from keras.models import Model
# from keras.layers import Dense, GlobalAveragePooling2D
# from keras import backend as K

In [ ]:
# batch_size = 32
# # tf.enable_eager_execution
# epochs = 5
# num_classes = 196

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_acc = []

    def on_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['acc'])

checkPoint = tf.keras.callbacks.ModelCheckpoint(filepath='./ece228_project/weights_incept.h5', save_weights_only=True, period=4)
batch_stats = CollectBatchStats()

In [ ]:
# base_model = InceptionV3(weights='imagenet', include_top=False)

# # add a global spatial average pooling layer
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# # and a logistic layer -- let's say we have 200 classes
# predictions = Dense(num_classes, activation='softmax')(x)

# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#     layer.trainable = False

# # compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# # train the model on the new data for a few epochs
# model.fit(ds,  epochs=epochs,\
#                   validation_data=None, verbose=1, steps_per_epoch=math.ceil(8144/32), callbacks=None) # [batch_stats, checkPoint]

# # at this point, the top layers are well trained and we can start fine-tuning
# # convolutional layers from inception V3. We will freeze the bottom N layers
# # and train the remaining top layers.

# # let's visualize layer names and layer indices to see how many layers
# # we should freeze:


In [ ]:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:249]:
#    layer.trainable = False
# for layer in model.layers[249:]:
#    layer.trainable = True

# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# from keras.optimizers import SGD
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers
# model.fit_generator(...)

### BASELINE


In [ ]:
model_path = './ece228_project'
batch_size = 32
# tf.enable_eager_execution
epochs = 200
num_classes = 196


In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_acc = []

    def on_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['acc'])

checkPoint = tf.keras.callbacks.ModelCheckpoint(filepath='./ece228_project/weights.h5', save_weights_only=True, period=99)
batch_stats = CollectBatchStats()

In [ ]:
class ConvBnReluBlock(tf.keras.Model):
    def __init__(self, filters, kernel, strides):
        super(ConvBnReluBlock, self).__init__()
        self.cnn = tf.keras.layers.Conv2D(filters, (kernel, kernel), strides=(strides, strides), kernel_initializer='he_normal',activation=tf.keras.activations.relu)
        self.bn = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=None, mask=None):
        x = self.cnn(inputs)
        x = self.bn(x)
        x = tf.nn.relu(x)
        return x

    

class CNN(tf.keras.Model):
    def __init__(self, num_classes=196):
        super(CNN, self).__init__()
        self.block1 = ConvBnReluBlock(32, kernel=5, strides=1)
        self.block2 = ConvBnReluBlock(64, kernel=5, strides=1)
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)
        self.pool2 = tf.keras.layers.GlobalAveragePooling2D()
        self.fullconnect = tf.keras.layers.Dense(1024)
        self.classifier = tf.keras.layers.Dense(num_classes)

    def call(self, inputs, training=None, mask=None):
        x = self.block1(inputs)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.pool2(x)
        x = self.fullconnect(x)
        output = self.classifier(x)

        # softmax op does not exist on the gpu, so always use cpu
        with tf.device('/cpu:0'):
            output = tf.nn.softmax(output)

        return output

device = '/gpu:0' 
# if tf.num_gpus() == 0 else '/gpu:0'

# with tf.device(device):
    # build model and optimizer


        

# evaluate on test set
# scores = model.evaluate(test_image_label_ds, batch_size=batch_size, verbose=1, steps=math.ceil(8144/32))
# print("Final test loss and accuracy :", scores)

In [ ]:
def train_model(train):
    dummy_x = tf.zeros((1 ,224, 224, 3))
    if train:
        model = CNN(num_classes)


        # TF Keras tries to use entire dataset to determine shape without this step when using .fit()
        # Fix = Use exactly one sample from the provided input dataset to determine input/output shape/s for the model
        
        model._set_inputs(dummy_x)

        model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # train
        model.fit(ds,  epochs=epochs,\
                  validation_data=None, verbose=1, steps_per_epoch=math.ceil(8144/32), callbacks=[batch_stats, checkPoint])
    else:
        test_model = CNN(num_classes)
        test_model._set_inputs(dummy_x)
        test_model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        test_model.load_weights(filepath='./ece228_project/weights.h5')
        scores = test_model.evaluate(test_image_label_ds, batch_size=batch_size, verbose=1, steps=tf.ceil(len(test_paths)/BATCH_SIZE).numpy().astype(np.int))
        print("Final test loss and accuracy :", scores)

In [ ]:
dummy_x = tf.zeros((1 ,224, 224, 3))

test_model = CNN(num_classes)
test_model._set_inputs(dummy_x)
test_model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
test_model.load_weights(filepath='./ece228_project/weights.h5')
scores = test_model.evaluate(test_image_label_ds, batch_size=batch_size, verbose=1, steps=tf.ceil(len(test_paths)/BATCH_SIZE).numpy().astype(np.int))
print("Final test loss and accuracy :", scores)

In [ ]:
tf.eagerly()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,10])
plt.plot(batch_stats.batch_losses)

plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats.batch_acc)